## Import Libraries

In [1]:
import argparse
import time
import numpy as np

from multiprocessing import Queue, Pipe

import brainflow
from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds
from brainflow.data_filter import DataFilter, FilterTypes, AggOperations

from pylsl import StreamInfo, StreamOutlet, local_clock

##  Configure Board Parameters with Brainflow

In [2]:
params = BrainFlowInputParams();
#params.ip_address = "192.168.4.1"; # for direct connect
params.ip_address = "192.168.0.100";
params.ip_port = 6677;
params.timeout = 0;
board_id = BoardIds.CYTON_DAISY_WIFI_BOARD.value;
board = BoardShim(board_id,params);
print("Sampling Rate: ", board.get_sampling_rate(BoardIds.CYTON_DAISY_WIFI_BOARD.value));
print("Device name: ", board.get_device_name(BoardIds.CYTON_DAISY_WIFI_BOARD.value));
#print(board.get_sampling_rate(BoardIds.CYTON_DAISY_WIFI_BOARD.value));
#print("Is prepared: ", board.is_prepared());

board.prepare_session()

Sampling Rate:  1000
Device name:  CytonDaisyWifi


BrainFlowError: BOARD_WRITE_ERROR:4 unable to prepare streaming session

## LSL Initialization 

In [ ]:
# LSL initialization
def channel_select(board, board_id, data_type): 
    switcher = { 
        'EXG': board.get_exg_channels(board_id),
        # can add more
    }
 
    return switcher.get(data_type, "error")

eeg_channels = BoardShim.get_eeg_channels(board_id)
chans = channel_select(board, board.board_id, 'EXG')
n_channels = len(eeg_channels)
#n_channels = 8;
srate = board.get_sampling_rate(board.board_id)
info = StreamInfo('OpenBCIEEG1', 'EXG', n_channels, srate, 'double64', 'OpenBCItestEEG1')
outlet = StreamOutlet(info)
fw_delay = 0

## Board Settings

### Channel Settings

In [ ]:
# send commands to the board for every channel. Cyton has 8 Channels. Here, we turn off every channel except for 1 and 8.
# This is here for testing purposes.
#https://docs.openbci.com/Cyton/CytonSDK/#channel-setting-commands
#board.config_board("x1000110X") #Lower the gain to 1x on channel 1
#board.config_board("x1061000X")
#board.config_board("x2161000X")
#board.config_board("x3161000X")
#board.config_board("x4161000X")
#board.config_board("x5161000X")
#board.config_board("x6161000X")
#board.config_board("x7161000X")
#board.config_board("x8060110X")
#board.config_board("d");
#board.config_board("j");
#board.config_board("c"); # only 8 channels instead of 16
board.config_board("x1060001X")
board.config_board("x2064001X")
board.config_board("x3161000X")
board.config_board("x4161000X")
board.config_board("x5161000X")
board.config_board("x6161000X")
board.config_board("x7161000X")
board.config_board("x8161000X")
board.config_board("xQ161000X")
board.config_board("xW161000X")
board.config_board("xE161000X")
board.config_board("xR161000X")
board.config_board("xT161000X")
board.config_board("xY161000X")
board.config_board("xU161000X")
board.config_board("xI161000X")

In [ ]:
# Set Channels to Test Signal, Temp, Short, Regular

#board.config_board("x1060001X")
#board.config_board("x2061001X")
#board.config_board("x3064001X")
#board.config_board("x4065001X")
#board.config_board("x5060001X")
#board.config_board("x6061001X")
#board.config_board("x7064001X")
#board.config_board("x8065001X")
#board.config_board("xQ060001X")
#board.config_board("xW061001X")
#board.config_board("xE064001X")
#board.config_board("xR065001X")
#board.config_board("xT060001X")
#board.config_board("xY061001X")
#board.config_board("xU064001X")
#board.config_board("xI065001X")

In [ ]:
#board.config_board("A") # 5min SD Card

### Lead of Detection Settings

In [ ]:
# z (Channel, PChan, NChan) Z
# 6nA at 31.5Hz
# https://docs.openbci.com/Cyton/CytonSDK/#leadoff-impedance-commands

board.config_board("z110Z")
#board.config_board("z210Z")
#board.config_board("z310Z")
#board.config_board("z410Z")
#board.config_board("z510Z")
#board.config_board("z610Z")
#board.config_board("z710Z")
#board.config_board("z810Z")
#board.config_board("zQ10Z")
#board.config_board("zW10Z")
#board.config_board("zE10Z")
#board.config_board("zR10Z")
#board.config_board("zT10Z")
#board.config_board("zY10Z")
#board.config_board("zU10Z")
#board.config_board("zI10Z")

## Start Streaming Data

### Configure Stream 

In [ ]:
# start stream
board.start_stream(450000) # 450,000 Max
time.sleep(1)

start_time = local_clock()
sent_samples = 0
#queue = Queue(maxsize = 5*srate)
queue = Queue(maxsize = 100000) # this is the max queue size. If it exceeds this, pause the loop
#queue.cancel_join_thread()

### Stream to LSL

In [ ]:
# read data with brainflow and send it via LSL
print("Now sending data...")
count = 0;
first = True
while True:
    data = board.get_board_data()[chans]
    print(data)
    if first:
        # Unchain Mode Set
        board.config_board("P");
        first = False;
    if data.size == 0:
        count = count + 1;
    else:
        count = 0;    
    if count == 10:
        print("Board disconnected");
        break;
    for i in range(len(data[0])):
        queue.put(data[:,i].tolist())
    elapsed_time = local_clock() - start_time
    required_samples = int(srate * elapsed_time) - sent_samples
    if required_samples > 0 and queue.qsize() >= required_samples:    
        mychunk = []
        for i in range(required_samples):
            mychunk.append(queue.get())
        stamp = local_clock() - fw_delay 
        outlet.push_chunk(mychunk)
        sent_samples += required_samples
    time.sleep(1)

In [ ]:
board.stop_stream();